### Competition is over but learning continues!
Entered the competition quite late, so could not implement all ideas in time.
But happy that I could learn a lot and so sharing this kernel.
Primarily this kernel is about:
* Data pipeline for LSTM model
* How we can have two inputs, one feeding to LSTM and other to linear regression kind of model after concatenating with output of LSTM
* Feature Engineering and a bit of EDA
* Model Validation based on result analysis
* And most important, null loss handling. I was repeatedly getting null loss as I trained the model. As it turned out, it was because I was initializing variables as empty instead of zeros.

### Importing Libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import os
import numpy as np
from tensorflow import keras
from keras import Sequential
from keras.layers import LSTM, Dense, Input, BatchNormalization, Flatten
import tensorflow as tf
base_path = "/kaggle/input/m5-forecasting-accuracy/"


### Thought I would use TPU, but finally did not.

In [ ]:
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

### Method to generate prediction

In [ ]:
def save_pred():
    submission_data = pd.read_csv(base_path + "sample_submission.csv")
    test_data1 = train_validation_data[train_validation_data["id"].isin(list(submission_data["id"].values))].copy()
    test_data2 = train_evaluation_data[train_evaluation_data["id"].isin(list(submission_data["id"].values))].copy()
    test_generator1 = DataGenerator(test_data1, for_test=True, y=False, max_time_series=1913)
    test_generator2 = DataGenerator(test_data2, for_test=True, y=False, max_time_series=1941)
    pred1 = model.predict(test_generator1)
    pred2 = model.predict(test_generator2)
    
    cols = list(submission_data.columns.values)
    cols.remove("id")

    test_data1 = test_data1[["id"]]
    df = pd.DataFrame(pred1) 
    df.columns = cols
    test_data1 = pd.concat([test_data1, df], axis=1)
    
    test_data2 = test_data2[["id"]]
    df = pd.DataFrame(pred2)
    df.columns = cols
    test_data2 = pd.concat([test_data2, df], axis=1)
    
    test_data_final = pd.concat([test_data1, test_data2])
    test_data_final.describe()
    
    return test_data_final

### LSTM model
Added tanh, when I was struggling with nan value. But realized it is beacause of data.

In [ ]:
x = tf.constant(np.array([[1,2,3],[4,5,6]]))
y = tf.constant(np.array([[2,3,4],[5,6,7]]))
tf.concat([x,y], axis=0)

In [ ]:
def build_model(n_features=6, loss=0.001):
    with strategy.scope():
       
        input_a = Input(shape=(28, n_features), name="time_series_data")
        input_b = Input(shape=(28, n_features-1), name="flat_data")
        lstm_layer = LSTM(128, return_sequences=True)(input_a)
        norm_layer = BatchNormalization()(lstm_layer)
        flat_layer = Flatten()(norm_layer)
        dense_layer_a = Dense(28, activation="relu")(flat_layer)
        #dense_layer_a = Flatten(dense_layer_a)
        print(dense_layer_a.shape)
        qty_layer = input_a[:,:,0]
        print(qty_layer.shape)
        arr = []
        for i in range(28):
            concat_layer = tf.concat([qty_layer[0,i:28], dense_layer_a[0,0:i]], axis=0)
            #print("concat",concat_layer.shape)
            arr.append(concat_layer)
        new_qty_data = tf.reshape(tf.concat(arr, axis=0),(-1,28,28))
        print(new_qty_data.shape)
        
        
        
        concat_layer = tf.concat([new_qty_data, input_b], axis=2)
        dense_layer_b = Dense(128, activation="relu")(concat_layer)
        output_layer  = Dense(1, activation="relu")(dense_layer_b)
            
        model = keras.Model(inputs=[input_a, input_b], outputs=output_layer)
        opt = keras.optimizers.Adam(learning_rate=loss)
        rmsprop = keras.optimizers.RMSprop(lr=0.00001, rho=0.9, epsilon=1e-08)
        model.compile(loss='mse', optimizer=opt, metrics=['mse'])
        return model

build_model()    

### Data Generator
It was very satisfying to build this data generator. Though with tensorflow 2.0, I should have used dataset.
So my intution was that if I have to predict qty for an item, I would look at the below:
1. Recent trend
2. Trend on special days like weekdays
3. Trend on month days
4. Trend on snap days
5. Trend on holidays 
6. Trend on day before holiday
Also helpful will be to see trends for 
1. Store as a whole
2. Category as a whole
3. Department as whole
4. State wise trends
5. Price fluctuations

I will need to see how the qty varied with these events and then knowing events for the future, try to predict the qty. 

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data, max_time_series, n_features=6, for_test=False, y=True):
        'Initialization'
        self.data = data
        self.batch_size = 1
        self.list_IDs = list(data["id"].values)
        self.n_time_series = 28
        self.n_pred = 28
        self.max_time_series = max_time_series
        self.tot_time_series = self.max_time_series - self.n_time_series -self.n_pred + 1
        self.n_features = n_features #len(type_columns) + len(name_columns) + 6
        self.for_test = for_test
        self.shuffle = False if for_test else True
        self.on_epoch_end()
        self.y = y

    def __len__(self):
        'Denotes the number of batches per epoch'
        num_data = 1 if not self.y else(0.01 if self.for_test else 2)
        return int(np.floor(len(self.list_IDs) * num_data))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def get_col_trend(self,col, all_qty_calendar_data, all_qty_row, calendar_row,calendar_y_row, X_i,X_flat_i, idx):
        #all_qty_calendar_data[col] = all_qty_calendar_data[col].fillna("")
        #calendar_row[col] = calendar_row[col].fillna("")
        #calendar_y_row[col] = calendar_y_row[col].fillna("")
        all_qty_row.columns = list(all_qty_calendar_data[col].values)
        df = all_qty_row.T.reset_index()
        df.columns = [col,"qty"]
        df_grp = df.groupby(col).mean()
        dict_weekday = df_grp.to_dict()["qty"]
        summary_data = list(calendar_row[col].map(dict_weekday).values)
        X_i[:,idx] = summary_data
        summary_data = list(calendar_y_row[col].map(dict_weekday).values)
        X_flat_i[:,idx-1] = summary_data
                
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        X = np.zeros((self.batch_size, self.n_time_series, self.n_features))
        X_flat = np.zeros((self.batch_size, self.n_pred, self.n_features-1))
        y = np.zeros((self.batch_size, self.n_pred), dtype=float)

        for i, ID in enumerate(list_IDs_temp):
            row = self.data[self.data["id"] == ID]
            start_day = row["start_day"].values[0]
            if self.for_test:
                if self.y:
                    start = self.max_time_series - self.n_time_series - self.n_pred + 1
                else:
                    start = self.max_time_series - self.n_time_series + 1
            else:
                start = np.random.randint(low=start_day, high=self.tot_time_series+1, size=1)[0]

            qty_cols = ["d_" + str(k) for k in range(start, start + self.n_time_series)]
            #if (not self.y):
            #    print(qty_cols)

            y_cols = ["d_" + str(k) for k in range(start + self.n_time_series, start + self.n_time_series+self.n_pred)]
            #if (not self.y):
            #    print(y_cols)
            
            all_qty_cols = ["d_" + str(k) for k in range(start_day, self.max_time_series+1)]
        
            X_i = np.zeros((self.n_time_series, self.n_features))
            X_flat_i = np.zeros((self.n_pred, self.n_features-1))
            
            calendar_row = calendar_data[calendar_data["d"].isin(qty_cols)]
            calendar_y_row = calendar_data[calendar_data["d"].isin(y_cols)]
            X_i[:,0] = row[qty_cols].values
            num_cols = 1
            if 1==2:
                X_i[:,num_cols] = calendar_row["weekend"].values
                state_id = row["state_id"].values[0]
                snap_col = "snap_" + state_id
                X_i[:,2] = calendar_row[snap_col].values
        
            
            all_qty_row = row[all_qty_cols]
            all_qty_calendar_data = calendar_data[calendar_data["d"].isin(all_qty_cols)]
            
            self.get_col_trend("wday", all_qty_calendar_data, all_qty_row, calendar_row,calendar_y_row, X_i, X_flat_i,1)
            self.get_col_trend("month", all_qty_calendar_data, all_qty_row, calendar_row,calendar_y_row, X_i, X_flat_i,2)
            self.get_col_trend("event_name_1", all_qty_calendar_data, all_qty_row, calendar_row,calendar_y_row, X_i,X_flat_i, 3)
            state_id = row["state_id"].values[0]
            snap_col = "snap_" + state_id
            self.get_col_trend(snap_col, all_qty_calendar_data, all_qty_row, calendar_row,calendar_y_row, X_i,X_flat_i, 4)
            self.get_col_trend("prv_event_name_1", all_qty_calendar_data, all_qty_row, calendar_row,calendar_y_row, X_i,X_flat_i, 5)
            X[i,:] = X_i
            X_flat[i,:] = X_flat_i
            
            if self.y:
                y[i,:] = row[y_cols].values
                if 1==2:
                    if min_qty > 0:
                        y[i,:] = y[i,:] - min_qty
                    if max_qty != min_qty:
                        y[i,:] = y[i,:] / (max_qty - min_qty)
        y = y if self.y else None
        
        return {"time_series_data":X, "flat_data":X_flat }, y

### Method for error analysis

In [ ]:

def get_sample_pred(n_features=3):
    submission_data = pd.read_csv(base_path + "sample_submission.csv")
    test_data1 = train_validation_data[train_validation_data["id"].isin(list(submission_data["id"].values))].copy().head(20)
    test_data2 = train_evaluation_data[train_evaluation_data["id"].isin(list(submission_data["id"].values))].copy().head(20)
    test_generator1 = DataGenerator(test_data1, n_features=n_features, for_test=True, y=False, max_time_series=1913)
    test_generator2 = DataGenerator(test_data2, n_features=n_features, for_test=True, y=False, max_time_series=1941)
    pred1 = model.predict(test_generator1)
    pred2 = model.predict(test_generator2)
    
    cols = list(submission_data.columns.values)
    cols.remove("id")

    test_data1 = test_data1[["id"]]
    df = pd.DataFrame(pred1) 
    df.columns = cols
    test_data1 = pd.concat([test_data1, df], axis=1)

    
    test_data2 = test_data2[["id"]]
    df = pd.DataFrame(pred2)
    df.columns = cols
    test_data2 = pd.concat([test_data2, df], axis=1)
    
    test_data_final = pd.concat([test_data1, test_data2])
    test_data_final.describe()
    return test_data1, test_data2

### Read Data

In [ ]:

train_validation_data = pd.read_csv(base_path + "sales_train_validation.csv")
train_evaluation_data = pd.read_csv(base_path + "sales_train_evaluation.csv")
calendar_data = pd.read_csv(base_path + "calendar.csv")
prices_data = pd.read_csv(base_path + "sell_prices.csv")


In [ ]:
all_qty_cols = [col for col in train_validation_data.columns.values if "d_" in col]
train_validation_data["start_day"] = train_validation_data.apply(lambda row: np.array(row[all_qty_cols]>0).argmax() + 1, axis=1)

all_qty_cols = [col for col in train_evaluation_data.columns.values if "d_" in col]
train_evaluation_data["start_day"] = train_evaluation_data.apply(lambda row: np.array(row[all_qty_cols]>0).argmax() + 1, axis=1)

### Data Analysis
Let us see days on which max sale happens. As we can see below, it is weekends and so this should be a feature

In [ ]:
calendar_data["prv_event_name_1"] = calendar_data["event_name_1"].shift(-1).fillna("")
calendar_data["event_name_1"] = calendar_data["event_name_1"].fillna("")

In [ ]:
all_qty_cols = ["d_" + str(k) for k in range(1, 1941+1)]
imp_days = list(train_evaluation_data[all_qty_cols].sum().sort_values().tail(10).index.values)
calendar_data[calendar_data["d"].isin(imp_days)]

### Visualizing the trend!

In [ ]:
all_qty_cols = ["d_" + str(k) for k in range(1, 1941+1)]
food_data = train_evaluation_data[train_evaluation_data["cat_id"]=="FOODS"]
imp_days = list(food_data[all_qty_cols].sum().sort_values().tail(5).index.values)
calendar_data[calendar_data["d"].isin(imp_days)]

In [ ]:
all_qty_cols = ["d_" + str(k) for k in range(1, 1941+1)]
hobbies_data = train_evaluation_data[train_evaluation_data["cat_id"]=="HOBBIES"]
imp_days = list(food_data[all_qty_cols].sum().sort_values().tail(5).index.values)
calendar_data[calendar_data["d"].isin(imp_days)]

In [ ]:
all_qty_cols = ["d_" + str(k) for k in range(1, 1941+1)]
household_data = train_evaluation_data[train_evaluation_data["cat_id"]=="HOUSEHOLD"]
imp_days = list(food_data[all_qty_cols].sum().sort_values().tail(5).index.values)
calendar_data[calendar_data["d"].isin(imp_days)]

In [ ]:
import matplotlib.pyplot as plt
plt.plot(list(range(1, 1941+1)), train_evaluation_data[all_qty_cols].sum().values)

In [ ]:
train_validation_data.head(1)

In [ ]:
train_evaluation_data.head(1)

In [ ]:
if 1==2:
    calendar_data["weekend"] = calendar_data["wday"].map(lambda x: 1 if x<=2 else 0)
    calendar_data.head(10)

### Instantiate train and validation data generators!

#Features for 28 day time period before and for days to be predicted
* Weekday qty diff trend
* Month qty diff trend
* event_name_1 qty diff trend
* snap qty diff trend

* Category Weekday qty diff trend
* Category Month qty diff trend
* Category event_name_1 qty diff trend
* Category snap qty diff trend

* Store Weekday qty diff trend
* Store Month qty diff trend
* Store event_name_1 qty diff trend
* Store snap qty diff trend

* State Weekday qty diff trend
* State Month qty diff trend
* State event_name_1 qty diff trend
* State snap qty diff trend

In [ ]:
training_generator = DataGenerator(train_validation_data, 1913,6)
validation_generator = DataGenerator(train_evaluation_data, 1941,6, for_test=True)
for X,y in validation_generator:
    print(X["time_series_data"].shape, y.shape)
    break

In [ ]:
model = build_model()
model_history = model.fit_generator(generator=training_generator,
                    validation_data=validation_generator, verbose=1, epochs=2)

### Get some predictions for result analysis

In [ ]:
submission_data = pd.read_csv(base_path + "sample_submission.csv")
test_data1 = train_validation_data[train_validation_data["id"].isin(list(submission_data["id"].values))].copy().tail(1)
   
test_generator1 = DataGenerator(test_data1, n_features=6, for_test=True, y=False, max_time_series=1913)

In [ ]:
for X,y in test_generator1:
    pred = model.predict([X["time_series_data"],X["flat_data"]])
    break

In [ ]:
model2 = keras.Model(inputs=model.inputs, outputs=model.layers[-3].output)

In [ ]:
for X,y in test_generator1:
    pred2 = model2.predict([X["time_series_data"],X["flat_data"]])
    break

In [ ]:
print(pred2[0][:,0:1])
print(pred2[0][:,27:28])

In [ ]:
test_data1.tail(1)

In [ ]:
item_id = "FOODS_3_827_WI_3_validation"

### Plot prediction and true values.

In [ ]:
row1= train_validation_data[train_validation_data["id"]==item_id]
row1

In [ ]:
row2 = train_evaluation_data[train_evaluation_data["id"]==item_id.replace("validation","evaluation")]
row2

In [ ]:
plt.figure(figsize=(20,4))
col = ["d_" + str(i+1) for i in range(1913)]
plt.scatter(list(range(1913))[-28:], list(row1[col].values[0])[-28:], marker="x")

col = ["d_" + str(i+1) for i in range(1941)]
plt.scatter(list(range(1942))[-28:], list(row2[col].values[0])[-28:])


col = ["F" + str(i+1) for i in range(28)]
for i in range(1):
    row3 = pred[0,i,:]
    plt.scatter(list(range(1942))[-28:], list(pred.flatten()))

### See if there are any special events/holidays days etc that if considered would have helped the model to train better

In [ ]:
col = ["d_" + str(i+1) for i in range(1914,1942)]
calendar_data[calendar_data["d"].isin(col)]

### See weekday trend

In [ ]:
col = ["d_" + str(i+1) for i in range(1941)]
plt.scatter(calendar_data["weekday"].values[-20:], list(row2[col].values[0])[-20:], c="g")


col = ["F" + str(i+1) for i in range(28)]
plt.scatter(calendar_data["weekday"].values[-20:], list(pred.flatten())[-20:], c="r")

In [ ]:
#save_pred().to_csv("submission.csv", index=False)

### Conclusion
Lots learnt, would have been much better if I would have entered the competition in time as would have been able to experiment with lot more ideas.
One idea that want to implement is to use differences as features instead of absolutes. 

### Please do share comments/ideas. This community is so wonderful. It is extremely motivating to be able to interact! 